# **Cennet Büşra HAKAY - 202113709007 / Biyoinformatiğe Giriş Dersi Final Ödevi**

**KULLANILAN VERİ SETLERİNİN LİNKLERİ VE HİKAYELERİ:**                                               

dog.fasta->https://www.ncbi.nlm.nih.gov/nuccore/AF013216.1?report=fasta                
**Hikayesi**: Bu veri seti, Myxococcus xanthus DK 1622 bakterisinin genetik yapısını incelemek için hazırlanmıştır. İçeriğinde, bakterinin metabolizması ve çevresel uyumunda önemli rol oynayan genler yer alır. Veri, 1997 yılında University of Idaho araştırmacıları tarafından toplanmış ve bakterinin biyolojik süreçlerini anlamaya yönelik çalışmalar için kullanılmıştır.
Veri setinde yer alan "Dog" ifadesi, genellikle bir organizmanın ya da bir genin bilimsel adından ziyade bir takma ad veya etiket olacak şekilde kullanılmıştır. Veri setinin tanımına göre, bu ifade belirli bir gen ya da gen kümesi için kullanılmıştır.

Ancak, bu "Dog" kelimesinin gerçek anlamı, veri setini sağlayan araştırmacılar veya projenin bağlamı ile doğrudan ilişkilidir. Myxococcus xanthus gibi bir bakterinin doğrudan köpeklerle ilişkisi bulunmamaktadır; bu yalnızca adlandırma konvansiyonlarına bağlıdır.

         
mushroom.fasta->https://www.ncbi.nlm.nih.gov/nuccore/NM_080495.6?report=fasta                             
**Hikayesi**: Bu veri, Drosophila melanogaster (meyve sineği) türüne ait bir genin, mushroom body defect (mud) geninin bir varyantına dair bilgi içermektedir. Bu gen, sinir sistemi ve özellikle öğrenme, hafıza gibi işlevlerle ilgili olan mantar cisimciklerinin gelişiminde rol oynar.

Veri, NCBI RefSeq projesi kapsamında kurumsal ve bireysel bilimsel katkılarla oluşturulmuştur. FlyBase gibi kaynaklardan gelen genom anotasyonları ve yüksek doğruluklu genomik verilerle sürekli güncellenmiştir. Genin hem genomik hem de transkript seviyesindeki detayları, ilgili araştırmalarla desteklenmiş ve biyoinformatik analizlere uygun şekilde organize edilmiştir.

In [18]:
import pandas as pd
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

**Eğitim boyunca gerekli olacak kütüphaneler projeye dahil edildi.**

In [19]:
def read_fasta(file_path):
    """
    Verilen FASTA dosyasını okur ve sadece dizilim satırlarını alır.

    Parameters:
    file_path (str): FASTA dosyasının yolu.

    Returns:
    List[str]: Dizilimlerin listesi.
    """
    with open(file_path, "r") as file:
        lines = file.readlines()

    sequences = []
    for line in lines:
        if not line.startswith(">"):  # Başlık satırlarını atlar.
            seq = line.strip()  # Dizilim satırını temizler.
            if len(seq) > 1:  # Boş ve çok kısa dizilimleri filtreler.
                sequences.append(seq)
    return sequences

**Bu fonksiyon (read_fasta), bir FASTA dosyasını okuyarak sadece dizilimleri bir liste halinde döndürür. Başlık satırlarını (>) atlar, her satırın başındaki ve sonundaki gereksiz boşlukları temizler. Boş veya çok kısa satırları filtreleyerek yalnızca geçerli dizilimleri saklar. Sonuçta, tüm dizilimleri bir liste olarak geri döndürür. Docstring içerisinde fonksiyonun nasıl çalıştığı belirtildi.**

In [20]:
dog_sequences = read_fasta('dog.fasta')
mushroom_sequences = read_fasta('mushroom.fasta')

**Fasta formatındaki veriler okundu.**

In [21]:
def kmer_frequencies(sequences, k=2):
    """
    Her dizilim için k-mer frekanslarını hesaplar.

    Parameters:
    sequences (List[str]): Dizilimlerin listesi.
    k (int): K-mer uzunluğu.

    Returns:
    List[Counter]: Her dizilim için k-mer frekanslarını tutan bir liste.
    """
    kmer_counts = []
    for seq in sequences:
        kmers = [seq[i:i+k] for i in range(len(seq) - k + 1)]
        kmer_count = Counter(kmers)  # K-mer frekansları
        kmer_counts.append(kmer_count)
    return kmer_counts

**Buradaki amaç sınıflandırmak için özellik çıkarmaktır. Biyolojik dizilimlerden k-mer frekanslarını hesaplar. Her bir dizilim için, uzunluğu k olan alt dizilimler (k-mer'ler) çıkarılır ve bu k-mer'lerin frekansları bir sayaç (Counter) ile kaydedilir. Sonuç olarak, her dizilimin k-mer frekanslarını içeren bir liste döner. Bu, dizilimlerden anlamlı sayısal özellikler elde etmek için kullanılır.**

In [22]:
kmer_data_dog = kmer_frequencies(dog_sequences, k=4)
kmer_data_mushroom = kmer_frequencies(mushroom_sequences, k=4)

**Frekanslar veri setleri üzerinde uygulanır. (k=4 yapılarak oranlara tekrar bakıldı.)**

In [23]:
all_kmers = set()
for kmer_count in kmer_data_dog + kmer_data_mushroom:
    all_kmers.update(kmer_count.keys())

**Veri setlerinde bulunan tüm benzersiz k-mer dizilimleri bir kümede toplandı. (Buradaki amaç ortak bir k-mer sözlüğü oluşturmak.)**

In [24]:
def get_feature_vector(kmer_counts, all_kmers):
    """
    K-mer frekanslarından sayısal özellik vektörleri oluşturur.

    Parameters:
    kmer_counts (List[Counter]): K-mer frekansları.
    all_kmers (Set[str]): Tüm k-merlerin kümesi.

    Returns:
    List[List[int]]: Özellik vektörlerinin listesi.
    """
    feature_vector = []
    for kmer_count in kmer_counts:
        vector = [kmer_count.get(kmer, 0) for kmer in all_kmers]
        feature_vector.append(vector)
    return feature_vector
X_dog = get_feature_vector(kmer_data_dog, all_kmers)
X_mushroom = get_feature_vector(kmer_data_mushroom, all_kmers)

**K-mer frekansları sayısal vektörlere dönüştürülür. K-mer'lerin frekansları, tüm benzersiz k-mer'lerin bulunduğu küme (all_kmers) ile eşleşerek bir vektör oluşturulur. Eğer bir k-mer dizilimde bulunmazsa, bu k-mer için değer 0 olarak kabul edilir.**

In [25]:
# Etiketler oluşturuldu. (sınıflandırma için.)
y = ['dog'] * len(dog_sequences) + ['mushroom'] * len(mushroom_sequences)

# Veri birleştirildi.
X = X_dog + X_mushroom

# Normalize edildi.
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [26]:
# Veriyi eğitim ve test olarak ayrıldı. (80% eğitim, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


**Veri dengesizliğini önlemek için SMOTE kullanıldı.**

In [28]:
param_grid = {
    'n_estimators': [100, 200, 300],  # Ağaç sayısı
    'max_depth': [10, 20, None],      # Maksimum derinlik
    'min_samples_split': [2, 5, 10], # Dallanma için minimum örnek sayısı
    'min_samples_leaf': [1, 2, 4],   # Bir yaprak düğümdeki minimum örnek sayısı
    'bootstrap': [True, False]       # Bootstrap örnekleme kullanımı
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=3,
    scoring='f1_weighted'
)

grid_search.fit(X_train_balanced, y_train_balanced)

model = grid_search.best_estimator_

**Random Forest modelinin hiperparametreleri GridSearch ile optimize edildi.
GridSearch ile en iyi model için kombinasyonlar denenir ve grid_search.best_estimator_ ile en iyi model seçilerek işlem oluşturulur.**

In [29]:
y_pred = model.predict(X_test)

print("Best parameters:", grid_search.best_params_)
report = classification_report(y_test, y_pred, output_dict=True)

Best parameters: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


**Test verisi üzerinde tahmin yapmak için eğitilen en iyi model kullanılır ve performans değerlendirmesi yapılır.**

In [30]:
# GridSearchCV sonuçları pandas DataFrame'e dönüştürüldü.
results = grid_search.cv_results_
df_results = pd.DataFrame(results)

# En iyi 5 sonuç gösterildi.
print("\nGridSearchCV Results:")
print(df_results[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']].head())



GridSearchCV Results:
                                              params  mean_test_score  \
0  {'bootstrap': True, 'max_depth': 10, 'min_samp...         0.969297   
1  {'bootstrap': True, 'max_depth': 10, 'min_samp...         0.969297   
2  {'bootstrap': True, 'max_depth': 10, 'min_samp...         0.972114   
3  {'bootstrap': True, 'max_depth': 10, 'min_samp...         0.960782   
4  {'bootstrap': True, 'max_depth': 10, 'min_samp...         0.966470   

   std_test_score  rank_test_score  
0        0.037635                7  
1        0.037635                7  
2        0.033671                1  
3        0.049640              115  
4        0.041617               37  


In [31]:
# Model performansı pandas DataFrame'e dönüştürüldü.
df_report = pd.DataFrame(report).transpose()

# Model performans raporu.
print("\nClassification Report:")
print(df_report)



Classification Report:
              precision    recall  f1-score    support
dog            0.978261  0.978261  0.978261  46.000000
mushroom       0.941176  0.941176  0.941176  17.000000
accuracy       0.968254  0.968254  0.968254   0.968254
macro avg      0.959719  0.959719  0.959719  63.000000
weighted avg   0.968254  0.968254  0.968254  63.000000


**Model performans raporuna göre, geliştirdiğim sınıflandırma modeli, hem "dog" hem de "mushroom" sınıfları için oldukça yüksek bir doğruluk ve dengeli bir performans sergilemiştir. Özellikle, %96.83'lük genel doğruluk oranı, modelin veri setindeki örnekleri başarılı bir şekilde ayırt edebildiğini göstermektedir.**

In [32]:
def kmer_frequencies_to_dataframe(kmer_counts, all_kmers):
    """
    K-mer frekanslarını pandas DataFrame olarak döndürür.

    Parameters:
    kmer_counts (List[Counter]): K-mer frekansları.
    all_kmers (Set[str]): Tüm k-merlerin kümesi.

    Returns:
    pd.DataFrame: K-mer frekansları tablosu.
    """
    feature_matrix = []
    all_kmers_list = list(all_kmers)

    for kmer_count in kmer_counts:
        feature_matrix.append([kmer_count.get(kmer, 0) for kmer in all_kmers_list])

    df = pd.DataFrame(feature_matrix, columns=all_kmers_list)
    return df

df_dog = kmer_frequencies_to_dataframe(kmer_data_dog, all_kmers)
df_mushroom = kmer_frequencies_to_dataframe(kmer_data_mushroom, all_kmers)

# İlk 5 satır
print("\nDog K-mer Frequencies:")
print(df_dog.head())

print("\nMushroom K-mer Frequencies:")
print(df_mushroom.head())


Dog K-mer Frequencies:
   ACTT  ATTC  GTCA  GCTG  AAGA  TACT  TACA  AACA  CATG  TCGA  ...  CCAG  \
0     0     0     0     0     0     0     0     0     1     0  ...     2   
1     1     0     0     3     0     0     0     0     0     0  ...     0   
2     0     1     0     2     0     1     0     0     0     0  ...     0   
3     0     0     0     1     0     0     0     0     1     0  ...     1   
4     0     0     0     0     0     0     0     0     0     1  ...     1   

   AGGT  CGCG  TGTG  TGTT  TTAC  ATGA  AAAC  CTTG  ATTA  
0     1     0     0     1     0     0     0     0     0  
1     0     2     0     0     0     0     0     0     0  
2     0     2     0     0     1     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     1     0     0     0     0     0     0  

[5 rows x 256 columns]

Mushroom K-mer Frequencies:
   ACTT  ATTC  GTCA  GCTG  AAGA  TACT  TACA  AACA  CATG  TCGA  ...  CCAG  \
0     0     0     1     0     0     0     

**K-mer frekansları pandas DataFrame formatına dönüştürülerek analiz için düzenli bir tablo haline getirildi.**